In [3]:
# importamos simpy
import simpy
# importamos random
import random

ModuleNotFoundError: No module named 'simpy'

In [4]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# Primera Simulación con 25 procesos

In [3]:
# creamos una lista para guardar los datos
lista = []

In [ ]:
# Creamos las variables a utilizar
RANDOM_SEED = 42
PROCESOS = 25  # Número de procesos
INTERVAL = 1  # Definido para el problema 
MEMORIA = 100 # Cpacidad de memoria de la RAM
CAPACIDAD = 1 # Capacidad de CPU 
INSTRUCCIONES_POR_TIEMPO = 3 # Instrucciones en el CPU 

In [ ]:
# Definimos una clase para el proceso que al final es un programa que entra al sistema
class Programa:
    def __init__(self, env, nombre, memoria, instrucciones):
        self.env = env
        self.memoria = memoria
        self.nombre = nombre
        self.instrucciones = instrucciones
        self.estado = 'new'
        self.tiempo_espera = 0
        self.tiempo_cpu = 0
    

# Definimos los procesos tanto sus instrucciones como el espacio de memoria que se generara
def procesador(env, memoria_ram):
    while True:
        for i in range(PROCESOS):
            # damos un tiempo random antes de llegar al cpu 
            yield env.timeout(random.expovariate(1/INTERVAL)) #Para retonar consecutivamente valores
            # generamos el espacio en la memoria  ram que tomará el proceso
            memoria = random.randint(1,10)
            nombre = i
            # generamos la cantidad de instrucciones totales a realizar por el process 
            instrucciones = random.randint(1,10)
            print('El proceso %g tiene un espacio de memoria de %f y tiene %d instrucciones'%(nombre, memoria,instrucciones))
            # Iniciamos a generar procesos
            proceso = Programa(env, nombre, memoria, instrucciones)
            env.process(interaccion(env, proceso, memoria_ram))

# Se generara la interaccion de new, ready y el paso al CPU
def interaccion(env, process, memoria_ram):
    process.estado = 'new'
    print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g '%(env.now))
    with memoria_ram.get(process.memoria):
        yield memoria_ram.get(process.memoria)
        process.estado = 'ready'
        print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g '%(env.now))
        env.process(cpu_proceso(env, process))
    memoria_ram.put(process.memoria)
    process.estado = 'terminated'

# Se define el funcionamiento del proceso del CPU
def cpu_proceso(env, process):
    with cpu.request() as req:
        yield req
        process.estado = 'running'
        print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g'%(env.now))
        while process.instrucciones > 0:
            yield env.timeout(1)
            process.instrucciones -= INSTRUCCIONES_POR_TIEMPO
            process.tiempo_cpu += 1
            if process.instrucciones <= 0:
                break
            if process.tiempo_cpu % 3 == 0:
                if random.randint(1, 2) == 1:
                    process.estado = 'waiting'
                    print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g '%(env.now))
                    env.process(io_proceso(env, process))
                    yield env.timeout(1)
                    process.tiempo_espera += 1
                else:
                    process.estado = 'ready'
                    print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g '%(env.now))
                    env.process(cpu_proceso(env, process))
        process.estado = 'terminated'
        print('El proceso %g termino con exito '%(process.nombre) + ' a las %g'%(env.now))

# Definimos el procedimiento para las operaciones de I/O
def io_proceso(env, process):
    print('El proceso %g entro a los procesos I/O '%(process.nombre)+' a las %g'%(env.now))
    yield env.timeout(random.randint(1, 2))
    process.estado = 'ready'
    print('El estado del proceso %g es '%(process.nombre)+ process.estado + ' a las %g '%(env.now))
    env.process(cpu_proceso(env, process))

: 

In [5]:
# creamos el ambiente 
env = simpy.Environment()
random.seed(RANDOM_SEED)
cpu = simpy.Resource(env, capacity = CAPACIDAD)
memoria_ram = simpy.Container(env, init = MEMORIA , capacity= MEMORIA)
env.process(procesador(env, memoria_ram )) 
env.run(until= PROCESOS)

In [6]:
# fijamos la cantidad de processs a simular 
for i in range(25):
    env.process(process('process numero %d'%i,env, random.expovariate(1/10),CPU))
    
env.run()

Programa numero 13 llega a las 0.455795 y requiere 8 instrucciones
Programa numero 22 llega a las 1.426911 y requiere 10 instrucciones
Programa numero 7 llega a las 1.746267 y requiere 7 instrucciones
Programa numero 3 llega a las 2.307955 y requiere 1 instrucciones
Programa numero 10 llega a las 2.876776 y requiere 10 instrucciones
Programa numero 17 llega a las 3.335420 y requiere 1 instrucciones
Programa numero 9 llega a las 3.971589 y requiere 4 instrucciones
Programa numero 16 llega a las 4.806295 y requiere 3 instrucciones
Programa numero 1 llega a las 5.601718 y requiere 4 instrucciones
Programa numero 19 llega a las 6.103351 y requiere 5 instrucciones
Programa numero 8 llega a las 7.353646 y requiere 9 instrucciones
Programa numero 13 sale del cpu a las 8.455795
Programa numero 0 llega a las 8.472372 y requiere 6 instrucciones
Programa numero 2 llega a las 8.629663 y requiere 4 instrucciones
Programa numero 15 llega a las 9.242992 y requiere 6 instrucciones
Programa numero 6 ll

In [7]:
datos = {'25 procesos':lista}

dataframe = pd.DataFrame(datos)

In [8]:
dataframe


,25 procesos
0,8.000000
1,17.028884
2,23.709528
3,24.147840
4,33.579019
5,34.120375
6,37.484206
7,39.649501
8,42.854077
9,47.352445


In [10]:
dataframe["25 procesos"].mean()

68.78959825209033

In [11]:
dataframe["25 procesos"].std()

37.19360415080694